# Explainer

### Coolab settings

In [1]:
#Load your files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/XAI\ \&\ Computational\ Cognitive\ Systems\ Written\ Assignment/Explainability\ Project

/content/drive/MyDrive/XAI & Computational Cognitive Systems Written Assignment/Explainability Project


In [3]:
!ls

archive_c		config.yaml		datasets.py	 models.py    train.py
BaselineCNN_5_ckpt.pth	coolab_explainer.ipynb	explainer.ipynb  __pycache__  utils.py


In [4]:
#!pip install shap

### Import libraries

In [5]:
import os
import torch
import torch.nn as nn
import torchvision
from datasets import ASL_C_Dataset
from models import BaselineCNN
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from utils import parse_arguments, read_settings, save_checkpoint, load_checkpoint
import numpy as np
import shap

In [6]:
# os.environ['https_proxy'] = "http://hpc-proxy00.city.ac.uk:3128"
device = torch.device('cpu')
if torch.cuda.is_available():
   device = torch.device('cuda')
print('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'BaselineCNN_Backbone'

cpu


In [7]:
# Read settings from the YAML file
settings = read_settings(config_path='config.yaml')

# Access and use the settings as needed
data_settings = settings.get('data', {})
model_settings = settings.get('model', {})
train_settings = settings.get('train', {})

In [8]:
# Initialize
asl_dataset = ASL_C_Dataset(img_size=data_settings['img_size'])
data_len = len(asl_dataset)
train_len = int(data_len*data_settings['train_size'])
test_len = int((data_len - train_len)/2)
val_len = data_len - train_len - test_len
asl_dataset_train, asl_dataset_test, asl_dataset_valid = random_split(asl_dataset, [train_len, test_len, val_len])

# test data loader
asl_testloader = DataLoader(asl_dataset_test, batch_size=1, shuffle=True)

# Baseline model for evaluating
baselinemodel = BaselineCNN(img_dim=data_settings['img_size'], num_classes=data_settings['num_output'], num_kernels=model_settings['num_kernels'],
                            num_filters1=model_settings['num_filters1'], num_filters2=model_settings['num_filters2'], num_hidden=model_settings['num_hidden'],
                            num_hidden2=model_settings['num_hidden2'], pooling_dim=model_settings['pooling_dim'], stride=model_settings['stride'], padding=model_settings['padding'],
                            stridepool=model_settings['stridepool'], paddingpool=model_settings['paddingpool'])
ckptfile = "BaselineCNN_5_ckpt.pth"
optimizer = torch.optim.Adam(list(baselinemodel.parameters()), lr = train_settings['learning_rate'])

load_checkpoint(baselinemodel, optimizer, ckptfile)

baselinemodel = baselinemodel.to(device)

In [9]:
# Background using training samples
asl_trainloader = DataLoader(asl_dataset_train, batch_size=5, shuffle=True)
background_images = []
for i, (images, _) in enumerate(asl_trainloader):
    background_images.append(images)
    if i == 20:  # Collect 20 batches
        break

background_np = torch.cat(background_images, dim=0).to(device) # Concatenate all collected images
print(background_np.shape)

torch.Size([105, 3, 320, 320])


In [ ]:
# Initialize the SHAP DeepExplainer
explainer = shap.DeepExplainer(baselinemodel, background_np)

In [ ]:
# Explaining
images, labels = next(iter(asl_testloader))
print(f"SHape im: {images.shape}")
labels_np = labels.numpy()
images_np = images.numpy()

# Explain predictions for the new images
shap_values = explainer.shap_values(images)

In [ ]:
pred = baselinemodel(images.to(device))
predicted_label = torch.argmax(pred).numpy()

In [ ]:
# Select the first image in the batch and transpose it to (height, width, channels)
image_to_plot = images_np[0].transpose(1, 2, 0)
label_to_plot = labels_np[0]
print(label_to_plot)
print(image_to_plot.shape)

# Since shap_values will also be in the form (batch_size, channels, height, width),
# you should select and transpose in similar fashion:
shap_values_to_plot = [sv[0].transpose(1, 2, 0) for sv in shap_values]
print(shap_values_to_plot[0].shape)
print(shap_values.shape)
print(shap_values[0,:,:,:,-2].shape)
#shap_values_to_plot = shap_values
# Plot the SHAP values
shap.image_plot(shap_values[0,:,:,:,-1].transpose(1,2,0), image_to_plot)

In [ ]:
import matplotlib.pyplot as plt
for i in range(shap_values.shape[-1]):
    shap_values_to_plot = shap_values[0,:,:,:,i].transpose(1,2,0)

    shap.image_plot(shap_values_to_plot, image_to_plot, show=False)
    plt.title(f"Class {i}")
    plt.suptitle(f"True Label: {label_to_plot} --- Predicted Label: {predicted_label}")
    plt.show()